In [ ]:
!pip install deepface

In [23]:
from deepface import DeepFace
import cv2
import numpy as np
import glob
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Dense, Flatten

In [11]:
import zipfile
import os

# Unzip the uploaded faces.zip file
with zipfile.ZipFile("faces.zip", 'r') as zip_ref:
    zip_ref.extractall("faces")

In [19]:
# List image files and read labels
faces_dir = "/content/faces"
labels_path = "/content/labels.txt"

image_files = sorted(glob.glob(os.path.join(faces_dir, "*.jpg")))

# Load labels from labels.txt
labels = []
with open(labels_path, 'r') as file:
    for line in file:
        parts = line.strip().split()
        label = int(parts[0])
        labels.append(label)

# Split images and labels into training and testing sets
train_files, test_files, y_train, y_test = train_test_split(
    image_files, labels, test_size=0.2, random_state=42, shuffle=True
)

train_files, val_files, y_train, y_val = train_test_split(
    train_files, y_train, test_size=0.1, random_state=42, shuffle=True
)

In [20]:
def preprocess_images(file_list):
    images = []
    for file in file_list:
        img = cv2.imread(file)

        # Resize to FaceNet512 input
        img = cv2.resize(img, (160, 160))

        # Normalize
        img = img.astype("float32") / 255.0

        images.append(img)
    return np.array(images)

# Preprocess the images
X_train = preprocess_images(train_files)
X_val = preprocess_images(val_files)
X_test = preprocess_images(test_files)

In [44]:
# Function to generate embeddings
def generate_embeddings(image_paths, labels):
    embeddings = []

    # Process each image and extract embeddings
    for path in image_paths:
        # Read and preprocess image
        img = cv2.imread(path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
        img = cv2.resize(img, (160, 160))  # Resize to FaceNet input size
        img = img.astype("float32") / 255.0  # Normalize image

        # Extract embeddings using DeepFace
        result = DeepFace.represent(img_path=path, model_name="Facenet512", enforce_detection=False)
        if result:
            embeddings.append(np.array(result[0]["embedding"]))
        else:
            print(f"Face not detected in {path}")

    return np.array(embeddings)


# Generate embeddings
X_train_embeddings = generate_embeddings(train_files, y_train)
X_val_embeddings = generate_embeddings(val_files, y_val)
X_test_embeddings = generate_embeddings(test_files, y_test)

print(f"Train embeddings shape: {X_train_embeddings.shape}")
print(f"Validation embeddings shape: {X_val_embeddings.shape}")
print(f"Test embeddings shape: {X_test_embeddings.shape}")

Train embeddings shape: (2880, 512)
Validation embeddings shape: (320, 512)
Test embeddings shape: (800, 512)


In [46]:
# Convert labels to NumPy arrays
y_train = np.array(y_train)
y_val = np.array(y_val)
y_test = np.array(y_test)

In [71]:
# Build a neural network classifier
classifier = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(512,)),  # Input layer for Facenet512 embeddings
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Binary output for smile/non-smile
])

learning_rate = 1e-5
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

# Compile the model with the custom learning rate
classifier.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = classifier.fit(X_train_embeddings, y_train,
                         epochs=60,
                         batch_size=32,
                         validation_data=(X_val_embeddings, y_val))

# Evaluate the model
test_loss, test_accuracy = classifier.evaluate(X_test_embeddings, y_test)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

Epoch 1/60
90/90 [==============================] - 2s 9ms/step - loss: 0.7598 - accuracy: 0.5260 - val_loss: 0.6834 - val_accuracy: 0.5969
Epoch 2/60
90/90 [==============================] - 1s 8ms/step - loss: 0.6716 - accuracy: 0.5983 - val_loss: 0.6289 - val_accuracy: 0.6438
Epoch 3/60
90/90 [==============================] - 1s 8ms/step - loss: 0.6271 - accuracy: 0.6486 - val_loss: 0.5966 - val_accuracy: 0.6844
Epoch 4/60
90/90 [==============================] - 1s 8ms/step - loss: 0.5964 - accuracy: 0.6844 - val_loss: 0.5780 - val_accuracy: 0.7188
Epoch 5/60
90/90 [==============================] - 1s 8ms/step - loss: 0.5814 - accuracy: 0.6955 - val_loss: 0.5662 - val_accuracy: 0.7281
Epoch 6/60
90/90 [==============================] - 1s 7ms/step - loss: 0.5631 - accuracy: 0.7111 - val_loss: 0.5571 - val_accuracy: 0.7312
Epoch 7/60
90/90 [==============================] - 0s 5ms/step - loss: 0.5503 - accuracy: 0.7260 - val_loss: 0.5514 - val_accuracy: 0.7312
Epoch 8/60
90/90 [==

In [72]:
# Save the trained model
classifier.save('/content/smile_non_smile_classifier.h5')